In [1]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
import math
import matplotlib.pyplot as plt
import yfinance as yf
import riskfolio.Portfolio as pf 
import statsmodels.api as sm

#Telechargement des donnees a partir de yahoo
yf.pdr_override()

In [2]:
#Collecte de donnees
actifs = ['^GSPC','XQQ.TO', 'RBOT.TO', 'XEG.TO', 'XST.TO', 'XEQT.TO', 'XUSR.TO', 'XCSR.TO', 'XDSR.TO', 'XRE.TO'] # XIT.TO RETIRÉ 10/01/2022, ESG all around GGRO.TO (includes fixed income)
actifs.sort()

#Dates
import datetime 
today = datetime.datetime.today()
yesterday = today - datetime.timedelta(days = 1)
tomorrow = today + datetime.timedelta(days = 1) 
tomorrow = tomorrow.strftime("%Y-%m-%d")

debut = '2016-01-01' #Creation de raccourçis
fin = tomorrow

In [3]:
#Téléchargement des données
donnees = yf.download(actifs, start = debut, end = fin)
donnees = donnees.loc[:,('Adj Close', slice(None))]
donnees.columns = actifs

display(donnees)

[*********************100%***********************]  10 of 10 completed


,RBOT.TO,XCSR.TO,XDSR.TO,XEG.TO,XEQT.TO,XQQ.TO,XRE.TO,XST.TO,XUSR.TO,^GSPC
Date,,,,,,,,,,
2016-01-04,NaN,NaN,NaN,8.899272,NaN,36.942875,11.096333,46.546566,NaN,2012.660034
2016-01-05,NaN,NaN,NaN,8.960348,NaN,37.305252,11.034861,46.430496,NaN,2016.709961
2016-01-06,NaN,NaN,NaN,8.620078,NaN,36.842201,10.996438,46.885132,NaN,1990.260010
2016-01-07,NaN,NaN,NaN,8.262362,NaN,35.835590,10.712112,46.594940,NaN,1943.089966
2016-01-08,NaN,NaN,NaN,8.279815,NaN,35.432941,10.727483,46.701340,NaN,1922.030029
...,...,...,...,...,...,...,...,...,...,...
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4796.560059
2022-01-04,36.209999,65.940002,55.200001,10.980000,27.93,127.300003,20.700001,77.150002,63.849998,4793.540039
2022-01-05,35.520000,65.440002,54.869999,11.040000,27.59,123.400002,20.330000,76.510002,62.610001,4700.580078


In [4]:
# création de fichiers csv et excel 
donnees.to_csv(r'C:\Users\nickg\OneDrive - Université Laval\Gestion de portefeuille\python\ETF Monitoring\Prix_monitoring.csv')#, sep=';', decimal=",") # Options utiles aux besoins 
#donnees.to_csv(r"C:\Users\nickg\OneDrive - Université Laval\Gestion de portefeuille\python\data.csv")#, sep=';', decimal=",") # Options utiles aux besoins 
donnees.to_excel(r'C:\Users\nickg\OneDrive - Université Laval\Gestion de portefeuille\python\ETF Monitoring\Prix_monitoring.xlsx', sheet_name='Prix_ajuste')

In [5]:
#Création de raccoucis pour le code
fichier_excel = r'C:\Users\nickg\OneDrive - Université Laval\Gestion de portefeuille\python\ETF Monitoring\Prix_monitoring.xlsx' #Changer lendroit du fichier. Simplifie puisque nous n'avons pas retrouver la direction du fichier à chaque fois que nous voulons lui faire référence
nom_feuille = 'Prix_ajuste' #Pareillement^^
Colonne_optionnelle = 'Date' #Colonne que nous ne voulons pas tout le temps utiliser et qui peut changer selon le téléchargement du fichier. Par exemple, Date ou Dates.


In [6]:
#Importation des prix
Donnees = pd.read_excel(fichier_excel, sheet_name=nom_feuille, index_col=Colonne_optionnelle)
display(Donnees)

,RBOT.TO,XCSR.TO,XDSR.TO,XEG.TO,XEQT.TO,XQQ.TO,XRE.TO,XST.TO,XUSR.TO,^GSPC
Date,,,,,,,,,,
2016-01-04,NaN,NaN,NaN,8.899272,NaN,36.942875,11.096333,46.546566,NaN,2012.660034
2016-01-05,NaN,NaN,NaN,8.960348,NaN,37.305252,11.034861,46.430496,NaN,2016.709961
2016-01-06,NaN,NaN,NaN,8.620078,NaN,36.842201,10.996438,46.885132,NaN,1990.260010
2016-01-07,NaN,NaN,NaN,8.262362,NaN,35.835590,10.712112,46.594940,NaN,1943.089966
2016-01-08,NaN,NaN,NaN,8.279815,NaN,35.432941,10.727483,46.701340,NaN,1922.030029
...,...,...,...,...,...,...,...,...,...,...
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4796.560059
2022-01-04,36.209999,65.940002,55.200001,10.980000,27.93,127.300003,20.700001,77.150002,63.849998,4793.540039
2022-01-05,35.520000,65.440002,54.869999,11.040000,27.59,123.400002,20.330000,76.510002,62.610001,4700.580078


In [7]:
#Création d'une liste des actifs sans la 'Dates', sera utile plus tard pour la pondération
liste_actifs = pd.read_excel(fichier_excel, sheet_name=nom_feuille) #ce que nous voulons importer du fichier excel
Actifs = [] #Liste avec 'Dates'
for i in liste_actifs: #Création de la boucle qui permet d'enregistrer les colonnes
	Actifs.append(i)
	
Actifs.remove(Colonne_optionnelle) #Nous enlevons 'Dates'
display(Actifs) #Affiche la liste
display(len(Actifs))

['RBOT.TO',
 'XCSR.TO',
 'XDSR.TO',
 'XEG.TO',
 'XEQT.TO',
 'XQQ.TO',
 'XRE.TO',
 'XST.TO',
 'XUSR.TO',
 '^GSPC']

10

In [8]:
#Calcul des rendements

R = donnees.pct_change().dropna()
display(R)

,RBOT.TO,XCSR.TO,XDSR.TO,XEG.TO,XEQT.TO,XQQ.TO,XRE.TO,XST.TO,XUSR.TO,^GSPC
Date,,,,,,,,,,
2020-05-06,-0.002868,0.006088,0.000000,-0.027254,-0.000515,0.006651,-0.004143,0.002116,0.004078,-0.006979
2020-05-07,-0.009108,0.001164,0.005926,0.019397,0.002060,0.012511,0.004161,-0.008770,0.005256,0.011505
2020-05-08,0.049347,-0.000930,0.008837,0.027484,0.012847,0.013189,0.015193,0.015075,0.010456,0.016872
2020-05-11,0.014292,0.009772,0.002920,-0.008231,0.007103,0.009044,-0.010884,0.029863,0.013406,0.000133
2020-05-12,-0.019091,-0.011751,0.004124,0.012448,-0.016625,-0.019690,-0.034388,-0.013793,0.000000,-0.020500
...,...,...,...,...,...,...,...,...,...,...
2022-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006374
2022-01-04,-0.006584,-0.006928,-0.000543,0.042735,0.001075,-0.008335,-0.008146,-0.013048,-0.013900,-0.000630
2022-01-05,-0.019055,-0.007583,-0.005978,0.005465,-0.012173,-0.030636,-0.017874,-0.008296,-0.019420,-0.019393


In [9]:
#Suppression des termes 'NaN' pour Donnees
#Méthode alternative
Donnees_nan = Donnees.dropna()
display(Donnees_nan)

,RBOT.TO,XCSR.TO,XDSR.TO,XEG.TO,XEQT.TO,XQQ.TO,XRE.TO,XST.TO,XUSR.TO,^GSPC
Date,,,,,,,,,,
2020-05-05,20.876257,42.169147,39.863049,4.571514,19.108490,72.880196,13.808419,61.400177,42.311848,2868.439941
2020-05-06,20.816383,42.425854,39.863049,4.446923,19.098658,73.364891,13.751204,61.530094,42.484386,2848.419922
2020-05-07,20.626780,42.475227,40.099277,4.533178,19.137997,74.282730,13.808419,60.990448,42.707664,2881.189941
2020-05-08,21.644648,42.435730,40.453617,4.657769,19.383860,75.262444,14.018217,61.909855,43.154224,2929.800049
2020-05-11,21.953999,42.850414,40.571724,4.619433,19.521544,75.943085,13.865640,63.758648,43.732731,2930.189941
...,...,...,...,...,...,...,...,...,...,...
2021-12-30,36.450001,66.400002,55.230000,10.530000,27.900000,128.369995,20.870001,78.169998,64.750000,4778.729980
2022-01-04,36.209999,65.940002,55.200001,10.980000,27.930000,127.300003,20.700001,77.150002,63.849998,4793.540039
2022-01-05,35.520000,65.440002,54.869999,11.040000,27.590000,123.400002,20.330000,76.510002,62.610001,4700.580078


In [10]:
 #Attribution des poids
    #Méthode 2, utile si ptf équipondérée

#Actifs_1.remove('^GSPC')
Equi_poids = np.array([1/len(Actifs)]*len(Actifs))
display(Equi_poids)
display(sum(Equi_poids))

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

0.9999999999999999

In [11]:
# CAPM, Alphas et Betas 
import statsmodels.api as sm

#Choix du taux sans risque et calcul de l'exces de rendement
rf = 1.26/100/252 #Attention à la périodicité
Exces = (R-rf) #soustraction du taux sans-risque à chaque données
display(Exces*100)

,RBOT.TO,XCSR.TO,XDSR.TO,XEG.TO,XEQT.TO,XQQ.TO,XRE.TO,XST.TO,XUSR.TO,^GSPC
Date,,,,,,,,,,
2020-05-06,-0.291802,0.603754,-0.005000,-2.730374,-0.056456,0.660058,-0.419347,0.206591,0.402779,-0.702941
2020-05-07,-0.915839,0.111376,0.587601,1.934657,0.200978,1.246060,0.411071,-0.882044,0.520553,1.145463
2020-05-08,4.929692,-0.097989,0.878656,2.743422,1.279685,1.313898,1.514347,1.502460,1.040620,1.682154
2020-05-11,1.424226,0.972206,0.286956,-0.828051,0.705302,0.899357,-1.093422,2.981266,1.335556,0.008308
2020-05-12,-1.914095,-1.180129,0.407425,1.239822,-1.667477,-1.974045,-3.443798,-1.384297,-0.005000,-2.055032
...,...,...,...,...,...,...,...,...,...,...
2022-01-03,-0.005000,-0.005000,-0.005000,-0.005000,-0.005000,-0.005000,-0.005000,-0.005000,-0.005000,0.632405
2022-01-04,-0.663441,-0.697770,-0.059316,4.268503,0.102529,-0.838522,-0.819567,-1.309844,-1.394964,-0.067962
2022-01-05,-1.910547,-0.763265,-0.602829,0.541452,-1.222330,-3.068630,-1.792444,-0.834552,-1.947048,-1.944276


In [12]:
Moyenne_exces = Exces.mean()
display(Moyenne_exces*100)

RBOT.TO    0.119555
XCSR.TO    0.099456
XDSR.TO    0.067508
XEG.TO     0.237106
XEQT.TO    0.080584
XQQ.TO     0.123214
XRE.TO     0.091034
XST.TO     0.049821
XUSR.TO    0.085805
^GSPC      0.113079
dtype: float64

In [13]:
#Calcul des variances journalieres des différents actifs
Var_Actifs_journalier= Exces.var()
display(Var_Actifs_journalier)

RBOT.TO    0.000175
XCSR.TO    0.000056
XDSR.TO    0.000071
XEG.TO     0.000586
XEQT.TO    0.000054
XQQ.TO     0.000183
XRE.TO     0.000107
XST.TO     0.000077
XUSR.TO    0.000086
^GSPC      0.000097
dtype: float64

In [14]:
#Calcul des betas
betas = Exces.cov()/Var_Actifs_journalier
display(betas) #Par exemple, ici, l'axe des x correspond au choix de l'indice du marche

,RBOT.TO,XCSR.TO,XDSR.TO,XEG.TO,XEQT.TO,XQQ.TO,XRE.TO,XST.TO,XUSR.TO,^GSPC
RBOT.TO,1.000000,1.077096,0.819133,0.194577,1.284381,0.635253,0.488514,0.306699,0.731720,0.866280
XCSR.TO,0.342877,1.000000,0.411371,0.112601,0.745093,0.323109,0.381006,0.316631,0.473983,0.493485
XDSR.TO,0.330070,0.520718,1.000000,0.087685,0.605090,0.228854,0.267225,0.175084,0.481772,0.365834
XEG.TO,0.651669,1.184666,0.728803,1.000000,1.530436,0.371370,1.054816,0.076532,0.454547,1.153484
XEQT.TO,0.398051,0.725391,0.465385,0.141620,1.000000,0.407258,0.400003,0.306744,0.526917,0.627750
XQQ.TO,0.663016,1.059359,0.592765,0.115731,1.371518,1.000000,0.444388,0.467666,0.988326,1.082555
XRE.TO,0.299032,0.732639,0.405944,0.192789,0.790059,0.260632,1.000000,0.221349,0.427380,0.548280
XST.TO,0.135429,0.439205,0.191863,0.010090,0.437048,0.197859,0.159674,1.000000,0.281752,0.248078
XUSR.TO,0.357894,0.728264,0.584788,0.066382,0.831583,0.463161,0.341493,0.312089,1.000000,0.550890
^GSPC,0.478153,0.855656,0.501118,0.190101,1.118022,0.572509,0.494391,0.310098,0.621677,1.000000


In [15]:
#definissions des betas
indice_marche = '^GSPC'
betas_im = betas[indice_marche]
display(betas_im)

RBOT.TO    0.866280
XCSR.TO    0.493485
XDSR.TO    0.365834
XEG.TO     1.153484
XEQT.TO    0.627750
XQQ.TO     1.082555
XRE.TO     0.548280
XST.TO     0.248078
XUSR.TO    0.550890
^GSPC      1.000000
Name: ^GSPC, dtype: float64

In [16]:
#Alpha                                          #0.016079
Alpha = Moyenne_exces-(betas_im*Moyenne_exces[indice_marche])
display(Alpha)

RBOT.TO    2.159675e-04
XCSR.TO    4.365355e-04
XDSR.TO    2.613955e-04
XEG.TO     1.066719e-03
XEQT.TO    9.598459e-05
XQQ.TO     8.001831e-06
XRE.TO     2.903547e-04
XST.TO     2.176863e-04
XUSR.TO    2.351085e-04
^GSPC      6.505213e-19
dtype: float64

In [17]:
#Alternativement
import statsmodels.api as sm

#Définissions des variables X et Y
X = Exces[indice_marche]
y = Exces['XCSR.TO']

#Ajout de la constante à la variable X
X1 = sm.add_constant(X)

#Création du model de regression 
model = sm.OLS(y, X1)

#Affichage
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                XCSR.TO   R-squared:                       0.422
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     314.3
Date:                Sun, 09 Jan 2022   Prob (F-statistic):           3.50e-53
Time:                        12:52:43   Log-Likelihood:                 1621.7
No. Observations:                 432   AIC:                            -3239.
Df Residuals:                     430   BIC:                            -3231.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.000      1.587      0.1

C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [18]:
#Automatisation du processus
#Création d'une liste
Actifs = [] #Liste avec 'TSX_Comp'
for i in Donnees: #Création de la boucle qui permet d'enregistrer les colonnes
	Actifs.append(i)
	
Actifs.remove(indice_marche) #Nous enlevons 'TSX_Comp'
display(Actifs) #Affiche la liste

['RBOT.TO',
 'XCSR.TO',
 'XDSR.TO',
 'XEG.TO',
 'XEQT.TO',
 'XQQ.TO',
 'XRE.TO',
 'XST.TO',
 'XUSR.TO']

In [19]:
#La logique est de venir grouper les étapes précédentes en une seule étape par substitution

#Nos variables
    #X = Exces[indice_marche]
    #y = Exces['CDiscretionary']
    #X1 = sm.add_constant(X)
    #model = sm.OLS(y, X1)
    #results = model.fit()
    #print(results.summary())

#X1 = sm.add_constant(Exces[indice_marche])

#model = sm.OLS(Exces['CDiscretionary'], sm.add_constant(Exces[indice_marche]))

#results = sm.OLS(Exces['CDiscretionary'], sm.add_constant(Exces[indice_marche])).fit()


print(((sm.OLS(Exces['XDSR.TO'], sm.add_constant(Exces[indice_marche]))).fit()).summary())

                            OLS Regression Results                            
Dep. Variable:                XDSR.TO   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.181
Method:                 Least Squares   F-statistic:                     96.53
Date:                Sun, 09 Jan 2022   Prob (F-statistic):           1.10e-20
Time:                        12:52:43   Log-Likelihood:                 1496.1
No. Observations:                 432   AIC:                            -2988.
Df Residuals:                     430   BIC:                            -2980.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.000      0.710      0.4

C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [20]:
# Automatisation
for a in Actifs:
   print((((sm.OLS(Exces[a], sm.add_constant(Exces[indice_marche]))).fit()).summary)()) #Il suffit d'ajouter un () à la suite de .summary

C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarnin

                            OLS Regression Results                            
Dep. Variable:                RBOT.TO   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     304.1
Date:                Sun, 09 Jan 2022   Prob (F-statistic):           6.90e-52
Time:                        12:52:43   Log-Likelihood:                 1371.5
No. Observations:                 432   AIC:                            -2739.
Df Residuals:                     430   BIC:                            -2731.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0002      0.000      0.440      0.6

C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [21]:
#Sauvegarde sous des variables: CAPM_"Actifs"
CAPM = {}
for a in Actifs:
    #CAPM["CAPM_{0}".format(a)] = (((sm.OLS(Exces[a], sm.add_constant(Exces[indice_marche]))).fit()).summary)()
    CAPM["CAPM_{0}".format(a)] = (((sm.OLS(Exces[a], sm.add_constant(Exces[indice_marche]))).fit()))

C:\Users\nickg\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [22]:
#Test
display(CAPM)

{'CAPM_RBOT.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8d2cc340>,
 'CAPM_XCSR.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8e3f3550>,
 'CAPM_XDSR.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8d303d90>,
 'CAPM_XEG.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8d303160>,
 'CAPM_XEQT.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8e4be9a0>,
 'CAPM_XQQ.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8e4beaf0>,
 'CAPM_XRE.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8d2b08e0>,
 'CAPM_XST.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8d2b0040>,
 'CAPM_XUSR.TO': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x15e8d2f1f40>}

In [23]:
#Si nous voulons faire référence à un seul actif en particulier
#print(CAPM['Actifs'])
print((CAPM['CAPM_XCSR.TO']).summary())
display((CAPM['CAPM_XCSR.TO']).rsquared)

                            OLS Regression Results                            
Dep. Variable:                XCSR.TO   R-squared:                       0.422
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     314.3
Date:                Sun, 09 Jan 2022   Prob (F-statistic):           3.50e-53
Time:                        12:52:43   Log-Likelihood:                 1621.7
No. Observations:                 432   AIC:                            -3239.
Df Residuals:                     430   BIC:                            -3231.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.000      1.587      0.1

0.4222533214027824